In [1]:
# !pip install --user spacy

import spacy
# # Load the medium english model. 
# # We will use this model to get embedding features for tokens later.

nlp = spacy.load('en_core_web_md', disable=['ner'])
nlp.remove_pipe('tagger')
nlp.remove_pipe('parser')

# Download a stopword list
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
#@Tokenize
def spacy_tokenize(string):
    tokens = list()
    doc = nlp(string)
    for token in doc:
        tokens.append(token)
    return tokens

#@Normalize
def normalize(tokens):
    normalized_tokens = list()
    for token in tokens:
        normalized = token.text.lower().strip()
        if ((token.is_alpha or token.is_digit)):
            normalized_tokens.append(normalized)
    return normalized_tokens

#@Tokenize and normalize
def tokenize_normalize(string):
    return normalize(spacy_tokenize(string))

In [8]:
tokenize_normalize("the app is fun. very happy.")

['the', 'app', 'is', 'fun', 'very', 'happy']

In [9]:
!pip install vaderSentiment

In [11]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sentences = ["VADER is smart, handsome, and funny.",  # positive sentence example
             "VADER is smart, handsome, and funny!",  # punctuation emphasis handled correctly (sentiment intensity adjusted)
             "VADER is very smart, handsome, and funny.", # booster words handled correctly (sentiment intensity adjusted)
             "VADER is VERY SMART, handsome, and FUNNY.",  # emphasis for ALLCAPS handled
             "VADER is VERY SMART, handsome, and FUNNY!!!", # combination of signals - VADER appropriately adjusts intensity
             "VADER is VERY SMART, uber handsome, and FRIGGIN FUNNY!!!", # booster words & punctuation make this close to ceiling for score
             "VADER is not smart, handsome, nor funny.",  # negation sentence example
             "The book was good.",  # positive sentence
             "At least it isn't a horrible book.",  # negated negative sentence with contraction
             "The book was only kind of good.", # qualified positive sentence is handled correctly (intensity adjusted)
             "The plot was good, but the characters are uncompelling and the dialog is not great.", # mixed negation sentence
             "Today SUX!",  # negative slang with capitalization emphasis
             "Today only kinda sux! But I'll get by, lol", # mixed sentiment example with slang and constrastive conjunction "but"
             "Make sure you :) or :D today!",  # emoticons handled
             "Catch utf-8 emoji such as such as 💘 and 💋 and 😁",  # emojis handled
             "Not bad at all"  # Capitalized negation
             ]

analyzer = SentimentIntensityAnalyzer()
for sentence in sentences:
    vs = analyzer.polarity_scores(sentence)
    print("{:-<65} {}".format(sentence, str(vs)))

VADER is smart, handsome, and funny.----------------------------- {'neg': 0.0, 'neu': 0.254, 'pos': 0.746, 'compound': 0.8316}
VADER is smart, handsome, and funny!----------------------------- {'neg': 0.0, 'neu': 0.248, 'pos': 0.752, 'compound': 0.8439}
VADER is very smart, handsome, and funny.------------------------ {'neg': 0.0, 'neu': 0.299, 'pos': 0.701, 'compound': 0.8545}
VADER is VERY SMART, handsome, and FUNNY.------------------------ {'neg': 0.0, 'neu': 0.246, 'pos': 0.754, 'compound': 0.9227}
VADER is VERY SMART, handsome, and FUNNY!!!---------------------- {'neg': 0.0, 'neu': 0.233, 'pos': 0.767, 'compound': 0.9342}
VADER is VERY SMART, uber handsome, and FRIGGIN FUNNY!!!--------- {'neg': 0.0, 'neu': 0.294, 'pos': 0.706, 'compound': 0.9469}
VADER is not smart, handsome, nor funny.------------------------- {'neg': 0.646, 'neu': 0.354, 'pos': 0.0, 'compound': -0.7424}
The book was good.----------------------------------------------- {'neg': 0.0, 'neu': 0.508, 'pos': 0.492, 'co

In [15]:
import pandas as pd
df_targets = pd.read_csv('Tweets_targets.csv')

In [16]:
len(df_targets['Tweets_text'])

14748

In [17]:
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import time

Emo_neg = np.zeros(len(df_targets['Tweets_text']))
Emo_neu = np.zeros(len(df_targets['Tweets_text']))
Emo_pos = np.zeros(len(df_targets['Tweets_text']))
Emo_compound = np.zeros(len(df_targets['Tweets_text']))

start = time.time()
sentences= df_targets['Tweets_text']
analyzer = SentimentIntensityAnalyzer()
for count,sentence in enumerate(sentences):
    vs = analyzer.polarity_scores(sentence)
#     print("{:-<65} {}".format(sentence, str(vs)))
#     print(vs['compound'])
    Emo_neg[count] = vs['neg'] 
    Emo_neu[count] = vs['neu'] 
    Emo_pos[count] = vs['pos'] 
    Emo_compound[count] = vs['compound']
end = time.time()
diff = end-start
print('Sentiment analysis took %d seconds' % (diff))
    

Sentiment analysis took 9 seconds


In [19]:
import pandas as pd
csv_input = pd.read_csv('Tweets_targets.csv')
csv_input['Emo_neg'] = Emo_neg
csv_input['Emo_neu'] = Emo_neu
csv_input['Emo_pos'] = Emo_pos
csv_input['Emo_compound'] = Emo_compound
csv_input.to_csv('Tweets_with_sentiment.csv', index=False)

In [23]:
# tweets_sentiment = pd.read_csv('Tweets_with_sentiment.csv')
tweets_sentiment[['Product','Tweets_text','Tweets_score']]
tweets_sentiment[['Tweets_text','Emo_neg','Emo_neu','Emo_pos','Emo_compound']]

,Tweets_text,Emo_neg,Emo_neu,Emo_pos,Emo_compound
0,"#5: Cheetah Mounts APTMM2B Flush Tilt (1.3"" Pr...",0.000,1.000,0.000,0.0000
1,okaaaay going to try &amp; go full vegan for 3...,0.000,1.000,0.000,0.0000
2,My summer CL used to stick her ear up to the d...,0.087,0.913,0.000,-0.3544
3,Don't stress over the things you can't control...,0.000,0.727,0.273,0.3867
4,After the 1948 Arabinvasion of Israel and for ...,0.000,1.000,0.000,0.0000
5,No one can control what happened in the past. ...,0.078,0.854,0.068,-0.0772
6,No one can control what happened in the past. ...,0.078,0.854,0.068,-0.0772
7,RT @der_bluthund: We Brits would like to apolo...,0.078,0.742,0.180,0.4404
8,"Money, Power &amp; greed ..kills more people t...",0.250,0.750,0.000,-0.8251
9,RT @ihatequotes: No one can control what happe...,0.078,0.854,0.068,-0.0772
